# Webscraping Instagram

## Connection use Selenium

In [17]:
#import libraries
from selenium import webdriver
import urllib.request
import pandas as pd
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import time

In [2]:
# Prepare the code for easily opening new browser window
driver_option = webdriver.ChromeOptions()
driver_option.add_argument(" — incognito")
chromedriver_path = '/home/igor/Estudos/Ciencia de Dados e Big Data/RI/chromedriver_linux'

def create_webdriver():
    return webdriver.Chrome(executable_path=chromedriver_path, options = driver_option)

In [42]:
#Open Google Chrome
driver = create_webdriver()
#Open page instagood
driver.get("https://www.instagram.com/explore/tags/instagood/")
time = 10
wait = WebDriverWait(driver, time)
wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]')))
photo = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[1]')
ActionChains(driver).click(photo).perform()

In [ ]:
#Capture URLs
instagram_url = []
for i in range(30000):
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//a[@class='HBoOv coreSpriteRightPaginationArrow']")))
        instagram_url.append(driver.current_url)
    except:
        ActionChains(driver).send_keys(Keys.ARROW_LEFT).perform()
    ActionChains(driver).send_keys(Keys.ARROW_RIGHT).perform()

In [51]:
pd_insta_url = pd.DataFrame(instagram_url, columns= ['urls'])
pd_insta_url.to_csv('instagram_urls.csv')

In [ ]:
#Change each photo
i = 0
while i <= 5000:
    print(i)
    try:
        wait.until(EC.presence_of_element_located((By.XPATH, "//h2[@class='BrX75']/a")))
        user = driver.find_element_by_xpath("//h2[@class='BrX75']/a").text
        print(user)
        wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='KL4Bh']/img")))
        image = driver.find_elements_by_xpath("//div[@class='KL4Bh']/img")
        if i >= len(image): 
            pos = len(image)-1
        else:
            pos = i
        if image[pos].get_attribute('alt')=='':
            description = 'video'
        else:
             description =  image[pos].get_attribute('alt')
        print(pos,': ',description)

    except:
        time +=1
        wait = WebDriverWait(driver, time)
    ActionChains(driver).send_keys(Keys.ARROW_RIGHT).perform()
    i+=1

In [38]:
image = driver.find_elements_by_xpath("//div[@class='KL4Bh']/img")
len(image)
image[len(image)-1].get_attribute('alt')

'Image may contain: dessert and food'

#### Check these
https://www.linkedin.com/pulse/scraping-instagram-selenuim-aman-khalid/

https://selenium-python.readthedocs.io/api.html#module-selenium.webdriver.common.action_chains

In [21]:
#Close connection
#driver.quit()